## Get (bugs) information from `.csv` files

The following part extracts information from the `.csv` files dumped during running test suites on DBMSs.

In [ ]:
import pandas as pd

results_df = pd.read_csv("output/mysql_results.csv")
logs_df = pd.read_csv("output/mysql_logs.csv")

print(results_df.info())
print(logs_df.info())


In [ ]:
from random import randint

bug_rows = results_df.loc[results_df.IS_ERROR == True]

print(bug_rows)
print(logs_df.LOGS)

bug_row = bug_rows.sample(1).squeeze()
dbms_name = bug_row.DBMS_NAME
log_row = logs_df.loc[bug_row.LOGS_INDEX]
test_suite = 'sqlite_tests'  # TODO same as above


## Reproduce an error testcase

The following code picks a bug record in the `.csv` file and run on the selected db. Maybe running using the testrunner.

In [ ]:
from src import testrunner

if dbms_name == 'sqlite':
    r = testrunner.SQLiteRunner()
elif dbms_name == 'duckdb':
    r = testrunner.DuckDBRunner()
elif dbms_name == 'cockroachdb':
    r = testrunner.CockroachDBRunner()
elif dbms_name == 'mysql':
    r = testrunner.MySQLRunner()
else:
    exit("Not implement yet")
    
r.connect("tempdb")

print(bug_row)
print(log_row)
print(log_row.LOGS)
r.executemany_stmt(log_row.LOGS)

print("The Fault SQL is %s" % bug_row.SQL)
print("The Expected Result is:")
print(bug_row.EXPECTED_RESULT)
print("The Actual Result is:")
result = r.execute_query(bug_row.SQL)
for row in result:
    for item in row:
        print(item)




